In [32]:
import xlrd
import json
import re
import time
from elasticsearch import Elasticsearch
es = Elasticsearch(port=9500)
def open_file(path):
    book = xlrd.open_workbook(path)
    return book.sheet_by_index(0)
    
typeNames = [
        "",
        "1 - Ensuring affected people, particularly women, have a stronger voice and greater role in humanitarian action",
        "2 - Confronting international humanitarian law (“IHL”) violations and finding new ways to protect and assist people in conflict",
        "3 - Localizing preparedness and response",
        "4 - Developing new approaches for managing recurrent and protected crises",
        "5 - Adapting the humanitarian system to new contexts, actors, and challenges",
        "6 - Global action to address the finance gap",
        "7 - Creating an enabling environment and investment in innovation to better deal with current and future humanitarian challenges",
        "8 - Gender"
    ];
#----------------------------------------------------------------------   
data = open_file('../WHSDATA.xlsx')
sentenceSplit = re.compile('(Pg\. ([0-9]{1,3})\/([0-9]{1,3})): ((.|.\n)*(\n\n|$))')

for row in range(2,479):
    doc = dict()
    doc['Name'] = data.cell(row,0).value
    texts = list()
    for col in range(70, 148):
        header = data.cell(0, col).value
        field = str.split(header, ":")
        title = field[0]
        if data.cell(row, col).value == 'x':
            if len(field) == 1:
                doc['Region'] = title.strip()
            else:
                doc[title] = field[1].strip()
    for col in range(1, 69):
        for sent in re.findall(sentenceSplit, data.cell(row, col).value):
            doc['page'] = int(sent[1])
            doc['total'] = int(sent[2])
            doc['text'] = sent[3]
            doc["subtopic"] = data.cell(0, col).value
            doc['topic'] = typeNames[int(doc["subtopic"][0])]
            es.index(index="whs6", doc_type='document', body=doc)
print("done")

done
